In [ ]:
import torch
from torch import nn
from tqdm.auto import tqdm
from torchvision import transforms
from torchvision.datasets import MNIST
from torchvision.utils import make_grid
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
torch.manual_seed(0) 

import pandas as pd
import random
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

import os

import numpy as np
import torch.nn.functional as F

from google.colab import drive
drive.mount('/content/drive')

from sklearn.metrics import accuracy_score
from datetime import datetime
from IPython.display import clear_output 

Mounted at /content/drive


In [ ]:
import os
import torch
os.environ['TORCH'] = torch.__version__
print(torch.__version__)

!pip install -q torch-scatter -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q torch-cluster -f https://data.pyg.org/whl/torch-${TORCH}.html
#!pip install torch-cluster -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
!pip install -q torch-sparse -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q git+https://github.com/pyg-team/pytorch_geometric.git

     |████████████████████████████████| 7.9 MB 6.1 MB/s 
     |████████████████████████████████| 3.5 MB 6.8 MB/s 
     |████████████████████████████████| 2.5 MB 4.2 MB/s 


In [ ]:

train_dataset=[]
graphlist=[]
import networkx as nx
import torch_geometric
from torch_geometric.data import Data, Dataset
from torch_geometric.loader import DataLoader
#from torch.utils.data import DataLoader
from torch_geometric.utils import degree
from torch_geometric.utils import erdos_renyi_graph, to_networkx, from_networkx
#%cd /content/drive/MyDrive/GNN_DATA/ADNI/ALL_NETWORKS
%cd /content/drive/MyDrive/ALL_NETWORKS

for i in range(1, 110):
    if os.path.isfile('AD_'+'{0:0>3}'.format(i)+'.csv'):
      tmp=pd.read_csv('AD_'+'{0:0>3}'.format(i)+'.csv', header=None)
      tmp=(tmp.to_numpy())
      tmp=torch.from_numpy(tmp)  
      tmp=torch.div(tmp,tmp.max())
      #print([tmp.type(torch.FloatTensor),torch.ones(1).type(torch.long)])
      G = nx.from_numpy_matrix(np.array(tmp))
      data = from_networkx(G)
      deg = degree(data.edge_index[0], data.num_nodes)
      x = []
      for i in deg:
        x.extend([i])
      #x = torch.tensor([[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1] ], dtype=torch.float)
      x = np.array(x).reshape(90,1)
      data.x = torch.tensor(x)
      data.y = torch.tensor([1])
      data.edge_attr = data.weight
      graphlist.append(data)
      #graphlist.append([tmp.type(torch.FloatTensor),torch.ones(1).type(torch.long)])

for i in range(1, 110):
    if os.path.isfile('CN_'+'{0:0>3}'.format(i)+'.csv'):
      tmp=pd.read_csv('CN_'+'{0:0>3}'.format(i)+'.csv', header=None)
      tmp=(tmp.to_numpy())
      tmp=torch.from_numpy(tmp) 
      tmp=torch.div(tmp, tmp.max())
      #print([tmp.type(torch.FloatTensor),torch.ones(1).type(torch.long)])
      G = nx.from_numpy_matrix(np.array(tmp))
      data = from_networkx(G)
      deg = degree(data.edge_index[0], data.num_nodes)
      x = []
      for i in deg:
        x.extend([i])
      #x = torch.tensor([[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1] ], dtype=torch.float)
      x = np.array(x).reshape(90,1)
      data.x = torch.tensor(x)
      data.y = torch.tensor([0])
      data.edge_attr = data.weight
      graphlist.append(data)
      #graphlist.append([data, torch.zeros(1).type(torch.long)])

      #graphlist.append([tmp.type(torch.FloatTensor),torch.zeros(1).type(torch.long)])

/content/drive/.shortcut-targets-by-id/10M6M6of9sCS0_CWq5d9EF6LhBCK9klNf/ALL_NETWORKS


In [ ]:
graphlist[0]

Data(edge_index=[2, 3814], weight=[3814], num_nodes=90, x=[90, 1], y=[1], edge_attr=[3814])

In [ ]:
%cd /content/drive/MyDrive
pos = pd.read_csv("node_aal90.csv", header = None)

/content/drive/MyDrive


In [ ]:
position1 = []
for i in pos[0]:
  position1.extend([i])
position1 = np.array(position1).reshape(90,1)

position2 = []
for i in pos[1]:
  position2.extend([i])
position2 = np.array(position2).reshape(90,1)

position3 = []
for i in pos[2]:
  position3.extend([i])
position3 = np.array(position3).reshape(90,1)

position = np.concatenate((position1, position2, position3), axis = 1) 

In [ ]:
for i in range(len(graphlist)):
  graphlist[i].x = torch.FloatTensor(np.concatenate((graphlist[i].x, position), axis = 1))

In [ ]:
dataset = random.sample(graphlist, len(graphlist))
train_dataset = dataset[:150]
test_dataset = dataset[150:]
train_loader = DataLoader(train_dataset, batch_size=25, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=25, shuffle=False)

In [ ]:
from torch_geometric.nn import GATConv, GraphConv
from torch_geometric.nn.conv.gatv2_conv import GATv2Conv
from torch_geometric.nn import global_mean_pool, global_sort_pool

class GNN(torch.nn.Module):
    def __init__(self):
        super(GNN, self).__init__()

        self.conv1 = GraphConv(in_channels=4, out_channels=16)
        self.conv2 = GraphConv(in_channels=16, out_channels=32)
        self.conv3 = GraphConv(in_channels=32, out_channels=32)
        self.lin1 = torch.nn.Linear(32, 16)
        self.lin2 = torch.nn.Linear(16, 1)
        self.sig = torch.nn.Sigmoid()

    def forward(self, data):
        x, edge_index, weight, batch = data.x, data.edge_index, data.weight, data.batch 
        x = self.conv1(x, edge_index, weight)
        x = F.relu(x)
        #print(x.size())
        x = self.conv2(x, edge_index, weight)
        x = F.relu(x)
        #print(x.size())
        x = self.conv3(x, edge_index, weight)
        x = F.relu(x)
        #print(x.size())
        x = global_mean_pool(x, batch)  # [batch_size, hidden_channels]
        #x = F.dropout(x, p=0.2, training=self.training)
        x = self.lin1(x)
        x = self.lin2(x)
        #print(x)
        
        return x
        #return F.log_softmax(x, dim=1)

In [ ]:
from IPython.display import Javascript
display(Javascript('''google.colab.output.setIframeHeight(0, true, {maxHeight: 300})'''))
sig=torch.nn.Sigmoid()
    
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GNN().to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = torch.nn.BCEWithLogitsLoss()
correct = 0

def test(loader):
     model.eval()

     correct = 0
     for data in loader:  # Iterate in batches over the training/test dataset.

         out = model(data.to(device))  
         pred = sig(out)>0.5  # Use the class with highest probability.
         #print(out.size())
         #print(int((pred == data.y).sum()))
         correct += int((pred.view(-1) == data.y).sum())
         #print(correct)  # Check against ground-truth labels.
     return correct / len(loader.dataset)  # Derive ratio of correct predictions.


for epoch in range(1, 100):
    model.train()

    for data in train_loader:  # Iterate in batches over the training dataset.
         out = model(data.to(device))
         out = out.view(-1)
         loss = criterion(out, data.y.float())  # Compute the loss.
         loss.backward()  # Derive gradients.
         optimizer.step()  # Update parameters based on gradients.
         optimizer.zero_grad()  # Clear gradients.

    train_acc = test(train_loader)
    test_acc = test(test_loader)
    print(f'Epoch: {epoch:03d}, Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f}, Loss: {loss:.4f}')

<IPython.core.display.Javascript object>

Epoch: 001, Train Acc: 0.5600, Test Acc: 0.3846, Loss: 1.7540
Epoch: 002, Train Acc: 0.4533, Test Acc: 0.5962, Loss: 0.7402
Epoch: 003, Train Acc: 0.5000, Test Acc: 0.6154, Loss: 0.8746
Epoch: 004, Train Acc: 0.5600, Test Acc: 0.4038, Loss: 0.8322
Epoch: 005, Train Acc: 0.4867, Test Acc: 0.6154, Loss: 0.7055
Epoch: 006, Train Acc: 0.5533, Test Acc: 0.5192, Loss: 0.6662
Epoch: 007, Train Acc: 0.5733, Test Acc: 0.4038, Loss: 0.8348
Epoch: 008, Train Acc: 0.5333, Test Acc: 0.6346, Loss: 0.6723
Epoch: 009, Train Acc: 0.5933, Test Acc: 0.4038, Loss: 0.6282
Epoch: 010, Train Acc: 0.6067, Test Acc: 0.5577, Loss: 0.7258
Epoch: 011, Train Acc: 0.6200, Test Acc: 0.5192, Loss: 0.6709
Epoch: 012, Train Acc: 0.6333, Test Acc: 0.5962, Loss: 0.6689
Epoch: 013, Train Acc: 0.6400, Test Acc: 0.4808, Loss: 0.6987
Epoch: 014, Train Acc: 0.6333, Test Acc: 0.5577, Loss: 0.6808
Epoch: 015, Train Acc: 0.6467, Test Acc: 0.5962, Loss: 0.6402
Epoch: 016, Train Acc: 0.6467, Test Acc: 0.5962, Loss: 0.6801
Epoch: 0

In [ ]:
import torch
import torch.nn.functional as F
from torch.nn import Linear

from torch_geometric.nn import GCNConv, JumpingKnowledge, global_mean_pool


class GCN(torch.nn.Module):
    def __init__(self):
        super(GCN, self).__init__()
        self.num_layers  = 3
        self.hidden = 16
        self.conv1 = GCNConv(4, self.hidden)
        self.convs = torch.nn.ModuleList()
        for i in range(self.num_layers - 1):
            self.convs.append(GCNConv(self.hidden, self.hidden))
        self.lin1 = Linear(self.hidden, self.hidden)
        self.lin2 = Linear(self.hidden, 1)

    def forward(self, data):
        x, edge_index,weight, batch = data.x, data.edge_index, data.weight, data.batch
        x = F.relu(self.conv1(x, edge_index, weight))
        for conv in self.convs:
            x = F.relu(conv(x, edge_index, weight))
        x = global_mean_pool(x, batch)
        x = F.relu(self.lin1(x))
        #x = F.dropout(x, p=0.5, training=self.training)
        x = self.lin2(x)
        return x

In [ ]:
from IPython.display import Javascript
display(Javascript('''google.colab.output.setIframeHeight(0, true, {maxHeight: 300})'''))
sig=torch.nn.Sigmoid()
    
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GCN().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = torch.nn.BCEWithLogitsLoss()
correct = 0

def test(loader):
     model.eval()

     correct = 0
     for data in loader:  # Iterate in batches over the training/test dataset.

         out = model(data.to(device))  
         pred = sig(out)>0.5  # Use the class with highest probability.
         #print(out.size())
         #print(int((pred == data.y).sum()))
         correct += int((pred.view(-1) == data.y).sum())
         #print(correct)  # Check against ground-truth labels.
     return correct / len(loader.dataset)  # Derive ratio of correct predictions.


for epoch in range(1, 500):
    model.train()

    for data in train_loader:  # Iterate in batches over the training dataset.
         out = model(data.to(device))
         out = out.view(-1)
         loss = criterion(out, data.y.float())  # Compute the loss.
         loss.backward()  # Derive gradients.
         optimizer.step()  # Update parameters based on gradients.
         optimizer.zero_grad()  # Clear gradients.

    train_acc = test(train_loader)
    test_acc = test(test_loader)
    print(f'Epoch: {epoch:03d}, Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f}, Loss: {loss:.4f}')

<IPython.core.display.Javascript object>

Epoch: 001, Train Acc: 0.4400, Test Acc: 0.6154, Loss: 0.7313
Epoch: 002, Train Acc: 0.5600, Test Acc: 0.3846, Loss: 0.6779
Epoch: 003, Train Acc: 0.5600, Test Acc: 0.3846, Loss: 0.6679
Epoch: 004, Train Acc: 0.5600, Test Acc: 0.3846, Loss: 0.6739
Epoch: 005, Train Acc: 0.5600, Test Acc: 0.3846, Loss: 0.6937
Epoch: 006, Train Acc: 0.5600, Test Acc: 0.3846, Loss: 0.7946
Epoch: 007, Train Acc: 0.5600, Test Acc: 0.3846, Loss: 0.6938
Epoch: 008, Train Acc: 0.5600, Test Acc: 0.3846, Loss: 0.6847
Epoch: 009, Train Acc: 0.5600, Test Acc: 0.3846, Loss: 0.6721
Epoch: 010, Train Acc: 0.5600, Test Acc: 0.3846, Loss: 0.6856
Epoch: 011, Train Acc: 0.5600, Test Acc: 0.3846, Loss: 0.6866
Epoch: 012, Train Acc: 0.5600, Test Acc: 0.3846, Loss: 0.7293
Epoch: 013, Train Acc: 0.5600, Test Acc: 0.3846, Loss: 0.7057
Epoch: 014, Train Acc: 0.5600, Test Acc: 0.3846, Loss: 0.6856
Epoch: 015, Train Acc: 0.5600, Test Acc: 0.3846, Loss: 0.6727
Epoch: 016, Train Acc: 0.5600, Test Acc: 0.3846, Loss: 0.6831
Epoch: 0

In [ ]:
from torch.nn import Linear
from torch_geometric.nn import GINConv, JumpingKnowledge, global_mean_pool
from torch.nn import BatchNorm1d as BN
from torch.nn import Linear, ReLU, Sequential
#from torch_geometric.nn import JumpingKnowledge, SAGEConv, global_mean_pool

class GIN(torch.nn.Module):
    def __init__(self):
        super(GIN, self).__init__()
        self.conv1 = GINConv(
            Sequential(
                Linear(4, 16),
                ReLU(),
                Linear(16, 16),
                ReLU(),
                BN(16),
            ), train_eps=False)
        self.conv2 = GINConv(
            Sequential(
                Linear(16, 32),
                ReLU(),
                Linear(32, 32),
                ReLU(),
                BN(32),
            ), train_eps=False)
        self.conv3 = GINConv(
            Sequential(
                Linear(32, 32),
                ReLU(),
                Linear(32, 32),
                ReLU(),
                BN(32),
            ), train_eps=False)
        self.lin1 = Linear(32, 16)
        self.lin2 = Linear(16, 1)

    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch
        x = self.conv1(x, edge_index)
        x = self.conv2(x, edge_index)
        x = self.conv3(x, edge_index)
        x = global_mean_pool(x, batch)
        x = F.relu(self.lin1(x))
        #x = F.dropout(x, p=0.5, training=self.training)
        x = self.lin2(x)
        return x
        #return F.log_softmax(x, dim=-1)

In [ ]:
from IPython.display import Javascript
display(Javascript('''google.colab.output.setIframeHeight(0, true, {maxHeight: 300})'''))
sig=torch.nn.Sigmoid()
    
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GIN().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = torch.nn.BCEWithLogitsLoss()
correct = 0

def test(loader):
     model.eval()

     correct = 0
     for data in loader:  # Iterate in batches over the training/test dataset.

         out = model(data.to(device))  
         pred = sig(out)>0.5  # Use the class with highest probability.
         #print(out.size())
         #print(int((pred == data.y).sum()))
         correct += int((pred.view(-1) == data.y).sum())
         #print(correct)  # Check against ground-truth labels.
     return correct / len(loader.dataset)  # Derive ratio of correct predictions.


for epoch in range(1, 200):
    model.train()

    for data in train_loader:  # Iterate in batches over the training dataset.
         out = model(data.to(device))
         out = out.view(-1)
         loss = criterion(out, data.y.float())  # Compute the loss.
         loss.backward()  # Derive gradients.
         optimizer.step()  # Update parameters based on gradients.
         optimizer.zero_grad()  # Clear gradients.

    train_acc = test(train_loader)
    test_acc = test(test_loader)
    print(f'Epoch: {epoch:03d}, Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f}, Loss: {loss:.4f}')

<IPython.core.display.Javascript object>

Epoch: 001, Train Acc: 0.4400, Test Acc: 0.6154, Loss: 0.7129
Epoch: 002, Train Acc: 0.4400, Test Acc: 0.6154, Loss: 0.6313
Epoch: 003, Train Acc: 0.4400, Test Acc: 0.6154, Loss: 0.6400
Epoch: 004, Train Acc: 0.5467, Test Acc: 0.6538, Loss: 0.6698
Epoch: 005, Train Acc: 0.5267, Test Acc: 0.6346, Loss: 0.6820
Epoch: 006, Train Acc: 0.4933, Test Acc: 0.5769, Loss: 0.5801
Epoch: 007, Train Acc: 0.5000, Test Acc: 0.6154, Loss: 0.5307
Epoch: 008, Train Acc: 0.6267, Test Acc: 0.4615, Loss: 0.6193
Epoch: 009, Train Acc: 0.4467, Test Acc: 0.5962, Loss: 0.4887
Epoch: 010, Train Acc: 0.5600, Test Acc: 0.5577, Loss: 0.6515
Epoch: 011, Train Acc: 0.6800, Test Acc: 0.4615, Loss: 0.7501
Epoch: 012, Train Acc: 0.5867, Test Acc: 0.5385, Loss: 0.5149
Epoch: 013, Train Acc: 0.6400, Test Acc: 0.5577, Loss: 0.7696
Epoch: 014, Train Acc: 0.7533, Test Acc: 0.4808, Loss: 0.5508
Epoch: 015, Train Acc: 0.5067, Test Acc: 0.6346, Loss: 0.4999
Epoch: 016, Train Acc: 0.6400, Test Acc: 0.5962, Loss: 0.4198
Epoch: 0

In [ ]:
import torch
import torch.nn.functional as F
from torch.nn import Linear

from torch_geometric.data import Batch
from torch_geometric.nn import (
    GraphConv,
    JumpingKnowledge,
    global_mean_pool,
    graclus,
    max_pool,
)

num_layers  = 4
hidden = 16
class Graclus(torch.nn.Module):
    def __init__(self):
        super(Graclus, self).__init__()
        self.conv1 = GraphConv(4, 16, aggr='mean')
        self.convs = torch.nn.ModuleList()
        for i in range(num_layers - 1):
            self.convs.append(GraphConv(16, 16, aggr='mean'))
        self.jump = JumpingKnowledge(mode='cat')
        self.lin1 = Linear(64, 16)
        self.lin2 = Linear(16, 1)

    def forward(self, data):
        x, edge_index, weight , batch = data.x, data.edge_index, data.weight, data.batch
        x = F.relu(self.conv1(x, edge_index, weight))
        xs = [global_mean_pool(x, batch)]
        for i, conv in enumerate(self.convs):
            x = F.relu(conv(x, edge_index))
            xs += [global_mean_pool(x, batch)]
            if i % 2 == 0 and i < len(self.convs) - 1:
                cluster = graclus(edge_index,num_nodes=x.size(0))
                data = Batch(x=x, edge_index=edge_index, batch=batch)
                data = max_pool(cluster, data)
                x, edge_index, batch = data.x, data.edge_index, data.batch
        x = self.jump(xs)
        x = F.relu(self.lin1(x))
        x = self.lin2(x)
        return x

In [ ]:
from IPython.display import Javascript
display(Javascript('''google.colab.output.setIframeHeight(0, true, {maxHeight: 300})'''))
sig=torch.nn.Sigmoid()
    
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Graclus().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = torch.nn.BCEWithLogitsLoss()
correct = 0

def test(loader):
     model.eval()

     correct = 0
     for data in loader:  # Iterate in batches over the training/test dataset.

         out = model(data.to(device))  
         pred = sig(out)>0.5  # Use the class with highest probability.
         #print(out.size())
         #print(int((pred == data.y).sum()))
         correct += int((pred.view(-1) == data.y).sum())
         #print(correct)  # Check against ground-truth labels.
     return correct / len(loader.dataset)  # Derive ratio of correct predictions.


for epoch in range(1, 500):
    model.train()

    for data in train_loader:  # Iterate in batches over the training dataset.
         out = model(data.to(device))
         out = out.view(-1)
         loss = criterion(out, data.y.float())  # Compute the loss.
         loss.backward()  # Derive gradients.
         optimizer.step()  # Update parameters based on gradients.
         optimizer.zero_grad()  # Clear gradients.

    train_acc = test(train_loader)
    test_acc = test(test_loader)
    print(f'Epoch: {epoch:03d}, Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f}, Loss: {loss:.4f}')

<IPython.core.display.Javascript object>

Epoch: 001, Train Acc: 0.5600, Test Acc: 0.3846, Loss: 0.6727
Epoch: 002, Train Acc: 0.5600, Test Acc: 0.3846, Loss: 0.6621
Epoch: 003, Train Acc: 0.5867, Test Acc: 0.4231, Loss: 0.6749
Epoch: 004, Train Acc: 0.4867, Test Acc: 0.5192, Loss: 0.6917
Epoch: 005, Train Acc: 0.5600, Test Acc: 0.3846, Loss: 0.6710
Epoch: 006, Train Acc: 0.5600, Test Acc: 0.3846, Loss: 0.7166
Epoch: 007, Train Acc: 0.5667, Test Acc: 0.4231, Loss: 0.6781
Epoch: 008, Train Acc: 0.5800, Test Acc: 0.4231, Loss: 0.7064
Epoch: 009, Train Acc: 0.5800, Test Acc: 0.4231, Loss: 0.6951
Epoch: 010, Train Acc: 0.5600, Test Acc: 0.3846, Loss: 0.7658
Epoch: 011, Train Acc: 0.6000, Test Acc: 0.4423, Loss: 0.6725
Epoch: 012, Train Acc: 0.5600, Test Acc: 0.3846, Loss: 0.8198
Epoch: 013, Train Acc: 0.5933, Test Acc: 0.4231, Loss: 0.6935
Epoch: 014, Train Acc: 0.6200, Test Acc: 0.4615, Loss: 0.6848
Epoch: 015, Train Acc: 0.5600, Test Acc: 0.3846, Loss: 0.6836
Epoch: 016, Train Acc: 0.5600, Test Acc: 0.3846, Loss: 0.6988
Epoch: 0

In [ ]:
import torch
import torch.nn.functional as F
from torch.nn import Linear

from torch_geometric.nn import (
    GraphConv,
    JumpingKnowledge,
    TopKPooling,
    global_mean_pool,
)


class TopK(torch.nn.Module):
    def __init__(self):
        super(TopK, self).__init__()
        self.ratio = 0.8
        self.num_layers = 3
        self.hidden = 16
        self.conv1 = GraphConv(4, 16, aggr='mean')
        self.convs = torch.nn.ModuleList()
        self.pools = torch.nn.ModuleList()
        self.convs.extend([
            GraphConv(16, 16, aggr='mean')
            for i in range(self.num_layers - 1)
        ])
        self.pools.extend(
            [TopKPooling(16, self.ratio) for i in range((self.num_layers) // 2)])
        self.jump = JumpingKnowledge(mode='cat')
        self.lin1 = Linear(48, 16)
        self.lin2 = Linear(16, 1)


    def forward(self, data):
        x, edge_index, weight, batch = data.x, data.edge_index, data.weight, data.batch
        x = F.relu(self.conv1(x, edge_index, weight))
        xs = [global_mean_pool(x, batch)]
        for i, conv in enumerate(self.convs):
            x = F.relu(conv(x, edge_index))
            xs += [global_mean_pool(x, batch)]
            if i % 2 == 0 and i < len(self.convs) - 1:
                pool = self.pools[i // 2]
                x, edge_index, _, batch, _, _ = pool(x, edge_index,
                                                     batch=batch)
        x = self.jump(xs)
        x = F.relu(self.lin1(x))
        #x = F.dropout(x, p=0.5, training=self.training)
        x = self.lin2(x)
        #return F.log_softmax(x, dim=-1)
        return x

In [ ]:
from IPython.display import Javascript
display(Javascript('''google.colab.output.setIframeHeight(0, true, {maxHeight: 300})'''))
sig=torch.nn.Sigmoid()
    
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = TopK().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = torch.nn.BCEWithLogitsLoss()
correct = 0

def test(loader):
     model.eval()

     correct = 0
     for data in loader:  # Iterate in batches over the training/test dataset.

         out = model(data.to(device))  
         pred = sig(out)>0.5  # Use the class with highest probability.
         #print(out.size())
         #print(int((pred == data.y).sum()))
         correct += int((pred.view(-1) == data.y).sum())
         #print(correct)  # Check against ground-truth labels.
     return correct / len(loader.dataset)  # Derive ratio of correct predictions.


for epoch in range(1, 300):
    model.train()

    for data in train_loader:  # Iterate in batches over the training dataset.
         out = model(data.to(device))
         out = out.view(-1)
         loss = criterion(out, data.y.float())  # Compute the loss.
         loss.backward()  # Derive gradients.
         optimizer.step()  # Update parameters based on gradients.
         optimizer.zero_grad()  # Clear gradients.

    train_acc = test(train_loader)
    test_acc = test(test_loader)
    print(f'Epoch: {epoch:03d}, Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f}, Loss: {loss:.4f}')

<IPython.core.display.Javascript object>

Epoch: 001, Train Acc: 0.5600, Test Acc: 0.3846, Loss: 0.6931
Epoch: 002, Train Acc: 0.5600, Test Acc: 0.3846, Loss: 0.7660
Epoch: 003, Train Acc: 0.5600, Test Acc: 0.3846, Loss: 0.7051
Epoch: 004, Train Acc: 0.5600, Test Acc: 0.4038, Loss: 0.6860
Epoch: 005, Train Acc: 0.5600, Test Acc: 0.3846, Loss: 0.6824
Epoch: 006, Train Acc: 0.5600, Test Acc: 0.3846, Loss: 0.6610
Epoch: 007, Train Acc: 0.5600, Test Acc: 0.3846, Loss: 0.6394
Epoch: 008, Train Acc: 0.5600, Test Acc: 0.3846, Loss: 0.6948
Epoch: 009, Train Acc: 0.5600, Test Acc: 0.3846, Loss: 0.6844
Epoch: 010, Train Acc: 0.5600, Test Acc: 0.3846, Loss: 0.6952
Epoch: 011, Train Acc: 0.5600, Test Acc: 0.3846, Loss: 0.7052
Epoch: 012, Train Acc: 0.5600, Test Acc: 0.3846, Loss: 0.6862
Epoch: 013, Train Acc: 0.5600, Test Acc: 0.3846, Loss: 0.7215
Epoch: 014, Train Acc: 0.5600, Test Acc: 0.3846, Loss: 0.6650
Epoch: 015, Train Acc: 0.5600, Test Acc: 0.3846, Loss: 0.7132
Epoch: 016, Train Acc: 0.5600, Test Acc: 0.3846, Loss: 0.7133
Epoch: 0

In [ ]:
import torch
import torch.nn.functional as F
from torch.nn import Linear

from torch_geometric.nn import (
    GraphConv,
    JumpingKnowledge,
    SAGPooling,
    global_mean_pool,
)


class SAGPool(torch.nn.Module):
    def __init__(self):
        super(SAGPool, self).__init__()
        self.num_layers = 4
        self.hidden = 32
        self.ratio = 0.8
        self.conv1 = GraphConv(4, self.hidden, aggr='mean')
        self.convs = torch.nn.ModuleList()
        self.pools = torch.nn.ModuleList()
        self.convs.extend([
            GraphConv(self.hidden, self.hidden, aggr='mean')
            for i in range(self.num_layers - 1)
        ])
        self.pools.extend(
            [SAGPooling(self.hidden, self.ratio) for i in range((self.num_layers) // 2)])
        self.jump = JumpingKnowledge(mode='cat')
        self.lin1 = Linear(self.num_layers * self.hidden, self.hidden)
        self.lin2 = Linear(self.hidden, 1)

    def forward(self, data):
        x, edge_index, weight, batch = data.x, data.edge_index, data.weight, data.batch
        x = F.relu(self.conv1(x, edge_index, weight))
        xs = [global_mean_pool(x, batch)]
        for i, conv in enumerate(self.convs):
            x = F.relu(conv(x, edge_index))
            xs += [global_mean_pool(x, batch)]
            if i % 2 == 0 and i < len(self.convs) - 1:
                pool = self.pools[i // 2]
                x, edge_index, _, batch, _, _ = pool(x, edge_index,
                                                     batch=batch)
        x = self.jump(xs)
        x = F.relu(self.lin1(x))
        #x = F.dropout(x, p=0.5, training=self.training)
        x = self.lin2(x)
        return x
        #return F.log_softmax(x, dim=-1)

In [ ]:
from IPython.display import Javascript
display(Javascript('''google.colab.output.setIframeHeight(0, true, {maxHeight: 300})'''))
sig=torch.nn.Sigmoid()
    
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = SAGPool().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = torch.nn.BCEWithLogitsLoss()
correct = 0

def test(loader):
     model.eval()

     correct = 0
     for data in loader:  # Iterate in batches over the training/test dataset.

         out = model(data.to(device))  
         pred = sig(out)>0.5  # Use the class with highest probability.
         #print(out.size())
         #print(int((pred == data.y).sum()))
         correct += int((pred.view(-1) == data.y).sum())
         #print(correct)  # Check against ground-truth labels.
     return correct / len(loader.dataset)  # Derive ratio of correct predictions.


for epoch in range(1, 300):
    model.train()

    for data in train_loader:  # Iterate in batches over the training dataset.
         out = model(data.to(device))
         out = out.view(-1)
         loss = criterion(out, data.y.float())  # Compute the loss.
         loss.backward()  # Derive gradients.
         optimizer.step()  # Update parameters based on gradients.
         optimizer.zero_grad()  # Clear gradients.

    train_acc = test(train_loader)
    test_acc = test(test_loader)
    print(f'Epoch: {epoch:03d}, Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f}, Loss: {loss:.4f}')

<IPython.core.display.Javascript object>

Epoch: 001, Train Acc: 0.5600, Test Acc: 0.3846, Loss: 0.8323
Epoch: 002, Train Acc: 0.4400, Test Acc: 0.6154, Loss: 0.7442
Epoch: 003, Train Acc: 0.5600, Test Acc: 0.3846, Loss: 0.6555
Epoch: 004, Train Acc: 0.5533, Test Acc: 0.3846, Loss: 0.6772
Epoch: 005, Train Acc: 0.5600, Test Acc: 0.3846, Loss: 0.7271
Epoch: 006, Train Acc: 0.5600, Test Acc: 0.3846, Loss: 0.6970
Epoch: 007, Train Acc: 0.5600, Test Acc: 0.3846, Loss: 0.7094
Epoch: 008, Train Acc: 0.5667, Test Acc: 0.3846, Loss: 0.6892
Epoch: 009, Train Acc: 0.5600, Test Acc: 0.3846, Loss: 0.6827
Epoch: 010, Train Acc: 0.5667, Test Acc: 0.4038, Loss: 0.7026
Epoch: 011, Train Acc: 0.5667, Test Acc: 0.3846, Loss: 0.6747
Epoch: 012, Train Acc: 0.5600, Test Acc: 0.3846, Loss: 0.6273
Epoch: 013, Train Acc: 0.5733, Test Acc: 0.4231, Loss: 0.6875
Epoch: 014, Train Acc: 0.5600, Test Acc: 0.3846, Loss: 0.6776
Epoch: 015, Train Acc: 0.6000, Test Acc: 0.4231, Loss: 0.6849
Epoch: 016, Train Acc: 0.6333, Test Acc: 0.4423, Loss: 0.6872
Epoch: 0

In [ ]:
import torch
import torch.nn.functional as F
from torch.nn import Linear

from torch_geometric.nn import (
    EdgePooling,
    GraphConv,
    JumpingKnowledge,
    global_mean_pool,
)


class EdgePool(torch.nn.Module):
    def __init__(self):
        super(EdgePool, self).__init__()
        self.num_layers = 3 
        self.hidden = 16
        self.conv1 = GraphConv(4, self.hidden, aggr='mean')
        self.convs = torch.nn.ModuleList()
        self.pools = torch.nn.ModuleList()
        self.convs.extend([
            GraphConv(self.hidden, self.hidden, aggr='mean')
            for i in range(self.num_layers - 1)
        ])
        self.pools.extend(
            [EdgePooling(self.hidden) for i in range((self.num_layers) // 2)])
        self.jump = JumpingKnowledge(mode='cat')
        self.lin1 = Linear(self.num_layers * self.hidden, self.hidden)
        self.lin2 = Linear(self.hidden, 1)

    def forward(self, data):
        x, edge_index, weight ,  batch = data.x, data.edge_index, data.weight, data.batch
        x = F.relu(self.conv1(x, edge_index, weight))
        xs = [global_mean_pool(x, batch)]
        for i, conv in enumerate(self.convs):
            x = F.relu(conv(x, edge_index))
            xs += [global_mean_pool(x, batch)]
            if i % 2 == 0 and i < len(self.convs) - 1:
                pool = self.pools[i // 2]
                x, edge_index, batch, _ = pool(x, edge_index, batch=batch)
        x = self.jump(xs)
        x = F.relu(self.lin1(x))
        #x = F.dropout(x, p=0.5, training=self.training)
        x = self.lin2(x)
        return x

In [ ]:
from IPython.display import Javascript
display(Javascript('''google.colab.output.setIframeHeight(0, true, {maxHeight: 300})'''))
sig=torch.nn.Sigmoid()
    
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = EdgePool().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = torch.nn.BCEWithLogitsLoss()
correct = 0

def test(loader):
     model.eval()

     correct = 0
     for data in loader:  # Iterate in batches over the training/test dataset.

         out = model(data.to(device))  
         pred = sig(out)>0.5  # Use the class with highest probability.
         #print(out.size())
         #print(int((pred == data.y).sum()))
         correct += int((pred.view(-1) == data.y).sum())
         #print(correct)  # Check against ground-truth labels.
     return correct / len(loader.dataset)  # Derive ratio of correct predictions.


for epoch in range(1, 100):
    model.train()

    for data in train_loader:  # Iterate in batches over the training dataset.
         out = model(data.to(device))
         out = out.view(-1)
         loss = criterion(out, data.y.float())  # Compute the loss.
         loss.backward()  # Derive gradients.
         optimizer.step()  # Update parameters based on gradients.
         optimizer.zero_grad()  # Clear gradients.

    train_acc = test(train_loader)
    test_acc = test(test_loader)
    print(f'Epoch: {epoch:03d}, Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f}, Loss: {loss:.4f}')

<IPython.core.display.Javascript object>

Epoch: 001, Train Acc: 0.4400, Test Acc: 0.6154, Loss: 0.7656
Epoch: 002, Train Acc: 0.5600, Test Acc: 0.3846, Loss: 0.8457
Epoch: 003, Train Acc: 0.4400, Test Acc: 0.6154, Loss: 0.7667
Epoch: 004, Train Acc: 0.5600, Test Acc: 0.3846, Loss: 0.7271
Epoch: 005, Train Acc: 0.5600, Test Acc: 0.3846, Loss: 0.6857
Epoch: 006, Train Acc: 0.5600, Test Acc: 0.3846, Loss: 0.6949
Epoch: 007, Train Acc: 0.5600, Test Acc: 0.3846, Loss: 0.7023
Epoch: 008, Train Acc: 0.5600, Test Acc: 0.3846, Loss: 0.7230
Epoch: 009, Train Acc: 0.5600, Test Acc: 0.3846, Loss: 0.6823
Epoch: 010, Train Acc: 0.5600, Test Acc: 0.3846, Loss: 0.6982
Epoch: 011, Train Acc: 0.5600, Test Acc: 0.3846, Loss: 0.6557
Epoch: 012, Train Acc: 0.5600, Test Acc: 0.3846, Loss: 0.6800
Epoch: 013, Train Acc: 0.5600, Test Acc: 0.3846, Loss: 0.7034
Epoch: 014, Train Acc: 0.5600, Test Acc: 0.3846, Loss: 0.6771
Epoch: 015, Train Acc: 0.5600, Test Acc: 0.3846, Loss: 0.6777
Epoch: 016, Train Acc: 0.5600, Test Acc: 0.3846, Loss: 0.7329
Epoch: 0

In [ ]:
import torch
import torch.nn.functional as F
from torch.nn import Linear

from torch_geometric.nn import GlobalAttention, SAGEConv


class GlobalAttentionNet(torch.nn.Module):
    def __init__(self):
        super(GlobalAttentionNet, self).__init__()
        self.num_layers = 3
        self.hidden  = 16
        self.conv1 = SAGEConv(4, self.hidden)
        self.convs = torch.nn.ModuleList()
        for i in range(self.num_layers - 1):
            self.convs.append(SAGEConv(self.hidden, self.hidden))
        self.att = GlobalAttention(Linear(self.hidden, 1))
        self.lin1 = Linear(self.hidden, self.hidden)
        self.lin2 = Linear(self.hidden, 1)

    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch
        x = F.relu(self.conv1(x, edge_index))
        for conv in self.convs:
            x = F.relu(conv(x, edge_index))
        x = self.att(x, batch)
        x = F.relu(self.lin1(x))
        #x = F.dropout(x, p=0.5, training=self.training)
        x = self.lin2(x)
        return x
        #return F.log_softmax(x, dim=-1)


In [ ]:
from IPython.display import Javascript
display(Javascript('''google.colab.output.setIframeHeight(0, true, {maxHeight: 300})'''))
sig=torch.nn.Sigmoid()
    
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GlobalAttentionNet().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
criterion = torch.nn.BCEWithLogitsLoss()
correct = 0

def test(loader):
     model.eval()

     correct = 0
     for data in loader:  # Iterate in batches over the training/test dataset.

         out = model(data.to(device))  
         pred = sig(out)>0.5  # Use the class with highest probability.
         #print(out.size())
         #print(int((pred == data.y).sum()))
         correct += int((pred.view(-1) == data.y).sum())
         #print(correct)  # Check against ground-truth labels.
     return correct / len(loader.dataset)  # Derive ratio of correct predictions.


for epoch in range(1, 300):
    model.train()

    for data in train_loader:  # Iterate in batches over the training dataset.
         out = model(data.to(device))
         out = out.view(-1)
         loss = criterion(out, data.y.float())  # Compute the loss.
         loss.backward()  # Derive gradients.
         optimizer.step()  # Update parameters based on gradients.
         optimizer.zero_grad()  # Clear gradients.

    train_acc = test(train_loader)
    test_acc = test(test_loader)
    print(f'Epoch: {epoch:03d}, Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f}, Loss: {loss:.4f}')

<IPython.core.display.Javascript object>

Epoch: 001, Train Acc: 0.4400, Test Acc: 0.6154, Loss: 0.9391
Epoch: 002, Train Acc: 0.4400, Test Acc: 0.6154, Loss: 1.0071
Epoch: 003, Train Acc: 0.4400, Test Acc: 0.6154, Loss: 1.0050
Epoch: 004, Train Acc: 0.4400, Test Acc: 0.6154, Loss: 0.9497
Epoch: 005, Train Acc: 0.4400, Test Acc: 0.6154, Loss: 0.8534
Epoch: 006, Train Acc: 0.4400, Test Acc: 0.6154, Loss: 0.7684
Epoch: 007, Train Acc: 0.4400, Test Acc: 0.6154, Loss: 0.7934
Epoch: 008, Train Acc: 0.4400, Test Acc: 0.6154, Loss: 0.7066
Epoch: 009, Train Acc: 0.4400, Test Acc: 0.6154, Loss: 0.7132
Epoch: 010, Train Acc: 0.5067, Test Acc: 0.5192, Loss: 0.6967
Epoch: 011, Train Acc: 0.6000, Test Acc: 0.3654, Loss: 0.6861
Epoch: 012, Train Acc: 0.5533, Test Acc: 0.3846, Loss: 0.6918
Epoch: 013, Train Acc: 0.5667, Test Acc: 0.3846, Loss: 0.6916
Epoch: 014, Train Acc: 0.5600, Test Acc: 0.3846, Loss: 0.7222
Epoch: 015, Train Acc: 0.5600, Test Acc: 0.3846, Loss: 0.6598
Epoch: 016, Train Acc: 0.5600, Test Acc: 0.3846, Loss: 0.7015
Epoch: 0

In [ ]:
import torch
import torch.nn.functional as F
from torch.nn import Linear

from torch_geometric.nn import SAGEConv, Set2Set


class Set2SetNet(torch.nn.Module):
    def __init__(self):
        super(Set2SetNet,self).__init__()
        self.hidden = 16
        self.num_layers = 3 
        self.conv1 = SAGEConv(4, self.hidden)
        self.convs = torch.nn.ModuleList()
        for i in range(self.num_layers - 1):
            self.convs.append(SAGEConv(self.hidden, self.hidden))
        self.set2set = Set2Set(self.hidden, processing_steps=4)
        self.lin1 = Linear(2 * self.hidden, self.hidden)
        self.lin2 = Linear(self.hidden, 1)

    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch
        x = F.relu(self.conv1(x, edge_index))
        for conv in self.convs:
            x = F.relu(conv(x, edge_index))
        x = self.set2set(x, batch)
        x = F.relu(self.lin1(x))
        #x = F.dropout(x, p=0.5, training=self.training)
        x = self.lin2(x)
        return x

In [ ]:
from IPython.display import Javascript
display(Javascript('''google.colab.output.setIframeHeight(0, true, {maxHeight: 300})'''))
sig=torch.nn.Sigmoid()
    
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Set2SetNet().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = torch.nn.BCEWithLogitsLoss()
correct = 0

def test(loader):
     model.eval()

     correct = 0
     for data in loader:  # Iterate in batches over the training/test dataset.

         out = model(data.to(device))  
         pred = sig(out)>0.5  # Use the class with highest probability.
         #print(out.size())
         #print(int((pred == data.y).sum()))
         correct += int((pred.view(-1) == data.y).sum())
         #print(correct)  # Check against ground-truth labels.
     return correct / len(loader.dataset)  # Derive ratio of correct predictions.


for epoch in range(1, 300):
    model.train()

    for data in train_loader:  # Iterate in batches over the training dataset.
         out = model(data.to(device))
         out = out.view(-1)
         loss = criterion(out, data.y.float())  # Compute the loss.
         loss.backward()  # Derive gradients.
         optimizer.step()  # Update parameters based on gradients.
         optimizer.zero_grad()  # Clear gradients.

    train_acc = test(train_loader)
    test_acc = test(test_loader)
    print(f'Epoch: {epoch:03d}, Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f}, Loss: {loss:.4f}')

<IPython.core.display.Javascript object>

Epoch: 001, Train Acc: 0.5600, Test Acc: 0.3846, Loss: 0.6810
Epoch: 002, Train Acc: 0.6200, Test Acc: 0.4615, Loss: 0.6999
Epoch: 003, Train Acc: 0.5600, Test Acc: 0.3846, Loss: 0.6881
Epoch: 004, Train Acc: 0.5667, Test Acc: 0.4038, Loss: 0.6661
Epoch: 005, Train Acc: 0.5733, Test Acc: 0.4038, Loss: 0.6929
Epoch: 006, Train Acc: 0.6467, Test Acc: 0.4423, Loss: 0.6700
Epoch: 007, Train Acc: 0.7067, Test Acc: 0.4615, Loss: 0.6097
Epoch: 008, Train Acc: 0.6933, Test Acc: 0.6538, Loss: 0.6815
Epoch: 009, Train Acc: 0.5733, Test Acc: 0.3846, Loss: 0.7382
Epoch: 010, Train Acc: 0.6867, Test Acc: 0.4615, Loss: 0.6463
Epoch: 011, Train Acc: 0.6800, Test Acc: 0.6346, Loss: 0.6086
Epoch: 012, Train Acc: 0.7000, Test Acc: 0.6923, Loss: 0.5427
Epoch: 013, Train Acc: 0.6867, Test Acc: 0.4808, Loss: 0.7978
Epoch: 014, Train Acc: 0.7333, Test Acc: 0.6538, Loss: 0.5935
Epoch: 015, Train Acc: 0.7533, Test Acc: 0.5962, Loss: 0.5696
Epoch: 016, Train Acc: 0.7200, Test Acc: 0.5577, Loss: 0.6934
Epoch: 0

In [ ]:
import torch
import torch.nn.functional as F
from torch.nn import Conv1d, Linear

from torch_geometric.nn import SAGEConv, global_sort_pool


class SortPool(torch.nn.Module):
    def __init__(self):
        super(SortPool, self).__init__()
        self.k = 30
        self.num_layers = 3 
        self.hidden = 16
        self.conv1 = SAGEConv(4, self.hidden)
        self.convs = torch.nn.ModuleList()
        for i in range(self.num_layers - 1):
            self.convs.append(SAGEConv(self.hidden, self.hidden))
        self.conv1d = Conv1d(self.hidden, 32, 5)
        self.lin1 = Linear(32 * (self.k - 5 + 1), self.hidden)
        self.lin2 = Linear(self.hidden, 1)

    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch
        x = F.relu(self.conv1(x, edge_index))
        for conv in self.convs:
            x = F.relu(conv(x, edge_index))
        x = global_sort_pool(x, batch, self.k)
        x = x.view(len(x), self.k, -1).permute(0, 2, 1)
        x = F.relu(self.conv1d(x))
        x = x.view(len(x), -1)
        x = F.relu(self.lin1(x))
        #x = F.dropout(x, p=0.5, training=self.training)
        x = self.lin2(x)
        return x

In [ ]:
from IPython.display import Javascript
display(Javascript('''google.colab.output.setIframeHeight(0, true, {maxHeight: 300})'''))
sig=torch.nn.Sigmoid()
    
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = SortPool().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
criterion = torch.nn.BCEWithLogitsLoss()
correct = 0

def test(loader):
     model.eval()

     correct = 0
     for data in loader:  # Iterate in batches over the training/test dataset.

         out = model(data.to(device))  
         pred = sig(out)>0.5  # Use the class with highest probability.
         #print(out.size())
         #print(int((pred == data.y).sum()))
         correct += int((pred.view(-1) == data.y).sum())
         #print(correct)  # Check against ground-truth labels.
     return correct / len(loader.dataset)  # Derive ratio of correct predictions.


for epoch in range(1, 300):
    model.train()

    for data in train_loader:  # Iterate in batches over the training dataset.
         out = model(data.to(device))
         out = out.view(-1)
         loss = criterion(out, data.y.float())  # Compute the loss.
         loss.backward()  # Derive gradients.
         optimizer.step()  # Update parameters based on gradients.
         optimizer.zero_grad()  # Clear gradients.

    train_acc = test(train_loader)
    test_acc = test(test_loader)
    print(f'Epoch: {epoch:03d}, Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f}, Loss: {loss:.4f}')

<IPython.core.display.Javascript object>

Epoch: 001, Train Acc: 0.4733, Test Acc: 0.5385, Loss: 0.7056
Epoch: 002, Train Acc: 0.5533, Test Acc: 0.3846, Loss: 0.7122
Epoch: 003, Train Acc: 0.5600, Test Acc: 0.3846, Loss: 0.7489
Epoch: 004, Train Acc: 0.5600, Test Acc: 0.3846, Loss: 0.6768
Epoch: 005, Train Acc: 0.5667, Test Acc: 0.3846, Loss: 0.6848
Epoch: 006, Train Acc: 0.5467, Test Acc: 0.3846, Loss: 0.7025
Epoch: 007, Train Acc: 0.5467, Test Acc: 0.3846, Loss: 0.6855
Epoch: 008, Train Acc: 0.5667, Test Acc: 0.3846, Loss: 0.6446
Epoch: 009, Train Acc: 0.5600, Test Acc: 0.3846, Loss: 0.6672
Epoch: 010, Train Acc: 0.5533, Test Acc: 0.3846, Loss: 0.6956
Epoch: 011, Train Acc: 0.5800, Test Acc: 0.3654, Loss: 0.6617
Epoch: 012, Train Acc: 0.5667, Test Acc: 0.3654, Loss: 0.6759
Epoch: 013, Train Acc: 0.5600, Test Acc: 0.3846, Loss: 0.6854
Epoch: 014, Train Acc: 0.5933, Test Acc: 0.3846, Loss: 0.6724
Epoch: 015, Train Acc: 0.5733, Test Acc: 0.3846, Loss: 0.6788
Epoch: 016, Train Acc: 0.6000, Test Acc: 0.3846, Loss: 0.6780
Epoch: 0

In [ ]:
import torch
import torch.nn.functional as F
from torch.nn import Linear

from torch_geometric.nn import (
    ASAPooling,
    GraphConv,
    JumpingKnowledge,
    global_mean_pool,
)


class ASAP(torch.nn.Module):
    def __init__(self):
        super(ASAP, self).__init__()
        self.num_layers = 5 
        self.hidden  = 32
        self.ratio = 0.8
        self.dropout = 0.2
        self.conv1 = GraphConv(4, self.hidden, aggr='mean')
        self.convs = torch.nn.ModuleList()
        self.pools = torch.nn.ModuleList()
        self.convs.extend([
            GraphConv(self.hidden, self.hidden, aggr='mean')
            for i in range(self.num_layers - 1)
        ])
        self.pools.extend([
            ASAPooling(self.hidden, self.ratio, dropout=self.dropout)
            for i in range((self.num_layers) // 2)
        ])
        self.jump = JumpingKnowledge(mode='cat')
        self.lin1 = Linear(self.num_layers * self.hidden, self.hidden)
        self.lin2 = Linear(self.hidden, 1)

    def forward(self, data):
        x, edge_index, edge_weight, batch = data.x, data.edge_index,data.weight, data.batch
        x = F.relu(self.conv1(x, edge_index, edge_weight))
        xs = [global_mean_pool(x, batch)]
        for i, conv in enumerate(self.convs):
            x = conv(x=x, edge_index=edge_index, edge_weight=edge_weight)
            x = F.relu(x)
            xs += [global_mean_pool(x, batch)]
            if i % 2 == 0 and i < len(self.convs) - 1:
                pool = self.pools[i // 2]
                x, edge_index, edge_weight, batch, _ = pool(
                    x=x, edge_index=edge_index, edge_weight=edge_weight,
                    batch=batch)
        x = self.jump(xs)
        x = F.relu(self.lin1(x))
        #x = F.dropout(x, p=0.5, training=self.training)
        x = self.lin2(x)
        return x
        #return F.log_softmax(x, dim=-1)


In [ ]:
from IPython.display import Javascript
display(Javascript('''google.colab.output.setIframeHeight(0, true, {maxHeight: 300})'''))
sig=torch.nn.Sigmoid()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = ASAP().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = torch.nn.BCEWithLogitsLoss()
correct = 0

def test(loader):
     model.eval()

     correct = 0
     for data in loader:  # Iterate in batches over the training/test dataset.

         out = model(data.to(device))  
         pred = sig(out)>0.5  # Use the class with highest probability.
         #print(out.size())
         #print(int((pred == data.y).sum()))
         correct += int((pred.view(-1) == data.y).sum())
         #print(correct)  # Check against ground-truth labels.
     return correct / len(loader.dataset)  # Derive ratio of correct predictions.


for epoch in range(1, 50):
    model.train()

    for data in train_loader:  # Iterate in batches over the training dataset.
         out = model(data.to(device))
         out = out.view(-1)
         loss = criterion(out, data.y.float())  # Compute the loss.
         loss.backward()  # Derive gradients.
         optimizer.step()  # Update parameters based on gradients.
         optimizer.zero_grad()  # Clear gradients.

    train_acc = test(train_loader)
    test_acc = test(test_loader)
    print(f'Epoch: {epoch:03d}, Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f}, Loss: {loss:.4f}')

<IPython.core.display.Javascript object>

Epoch: 001, Train Acc: 0.5600, Test Acc: 0.3846, Loss: 0.6963
Epoch: 002, Train Acc: 0.5600, Test Acc: 0.3846, Loss: 0.7432
Epoch: 003, Train Acc: 0.4400, Test Acc: 0.6154, Loss: 0.6991
Epoch: 004, Train Acc: 0.5600, Test Acc: 0.3846, Loss: 0.6574
Epoch: 005, Train Acc: 0.5600, Test Acc: 0.3846, Loss: 0.6693
Epoch: 006, Train Acc: 0.5600, Test Acc: 0.3846, Loss: 0.6949
Epoch: 007, Train Acc: 0.5600, Test Acc: 0.3846, Loss: 0.6725
Epoch: 008, Train Acc: 0.5600, Test Acc: 0.3846, Loss: 0.6810
Epoch: 009, Train Acc: 0.5600, Test Acc: 0.3846, Loss: 0.6829
Epoch: 010, Train Acc: 0.5600, Test Acc: 0.3846, Loss: 0.6963
Epoch: 011, Train Acc: 0.4467, Test Acc: 0.6154, Loss: 0.6904
Epoch: 012, Train Acc: 0.5733, Test Acc: 0.4231, Loss: 0.7166
Epoch: 013, Train Acc: 0.6133, Test Acc: 0.4423, Loss: 0.6613
Epoch: 014, Train Acc: 0.5600, Test Acc: 0.3846, Loss: 0.7577
Epoch: 015, Train Acc: 0.5933, Test Acc: 0.4231, Loss: 0.7076
Epoch: 016, Train Acc: 0.5600, Test Acc: 0.3846, Loss: 0.6894
Epoch: 0